# Identify Target Areas for Covid Relief

This notebook utilizes census data, covid case rates, covid vaccine rates, covid vaccine hesitancy information and information on Historically Black Colleges and Universities (HBCUs) to rank counties by need of covid relief for black americans. This analysis was used in determining which HBCUs should be targeted for a compaign to provide covid releif for black americans through vaccinations and education by ranking counties containing HBCUs by need. The analysis considers black population rates, overall population density, vaccine hesitancy rates and vaccine rates in ranking counties.

In [1]:
#import needed packages
import pandas as pd
import warnings; warnings.simplefilter('ignore')
import numpy as np
from google.cloud import bigquery

In [2]:
#connect to bigquery
client = bigquery.Client()

#run query and return dataframe
def run_query(query):
    query_result = client.query(query)
    return query_result.to_dataframe()

In [3]:
#pull in census data
census_query = '''SELECT geo_id, total_pop, black_pop
FROM `bigquery-public-data.census_bureau_acs.county_2018_5yr`
'''

census = run_query(census_query)
census.head()

,geo_id,total_pop,black_pop
0,35039,39307.0,164.0
1,72133,22066.0,0.0
2,72043,39265.0,8.0
3,72151,34149.0,0.0
4,72071,42420.0,98.0


In [4]:
#vaccination rate data
counties = pd.read_csv('counties.csv', dtype={0:'str'})
counties = counties[['fips', 'state','county', 'population', 'actuals.vaccinationsCompleted', 'metrics.vaccinationsCompletedRatio']]
counties.head()

,fips,state,county,population,actuals.vaccinationsCompleted,metrics.vaccinationsCompletedRatio
0,02013,AK,Aleutians East Borough,3337,2228.0,0.668
1,02016,AK,Aleutians West Census Area,5634,2923.0,0.519
2,02020,AK,Anchorage Municipality,288000,129683.0,0.450
3,02050,AK,Bethel Census Area,18386,9226.0,0.502
4,02060,AK,Bristol Bay Borough,836,712.0,0.852


In [5]:
#vaccine hesitancy data
vax_hes = pd.read_csv('cleaned_vaccine_0708.csv')
vax_hes = vax_hes[['final_zip_pred', 'state_name','county_name']]
vax_hes = vax_hes.groupby(['county_name', 'state_name'])['final_zip_pred'].mean().reset_index(name ='vax_hes')
vax_hes.head()

,county_name,state_name,vax_hes
0,Abbeville County,South Carolina,0.198282
1,Acadia Parish,Louisiana,0.247609
2,Accomack County,Virginia,0.144231
3,Ada County,Idaho,0.102163
4,Adair County,Iowa,0.169954


In [6]:
#map state names to abbreviations for joining data sets
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
vax_hes['state'] = vax_hes['state_name'].map(us_state_abbrev)

In [7]:
#join all the data sets
df_rank = pd.merge(counties, census, how='left', left_on='fips', right_on='geo_id')
df_rank = pd.merge(df_rank, vax_hes, how='left', left_on=['county', 'state'], right_on=['county_name', 'state'])
df_rank.head()

,fips,state,county,population,actuals.vaccinationsCompleted,metrics.vaccinationsCompletedRatio,geo_id,total_pop,black_pop,county_name,state_name,vax_hes
0,02013,AK,Aleutians East Borough,3337,2228.0,0.668,02013,3425.0,117.0,Aleutians East Borough,Alaska,0.114048
1,02016,AK,Aleutians West Census Area,5634,2923.0,0.519,02016,5750.0,246.0,Aleutians West Census Area,Alaska,0.122040
2,02020,AK,Anchorage Municipality,288000,129683.0,0.450,02020,296112.0,15671.0,Anchorage Municipality,Alaska,0.112535
3,02050,AK,Bethel Census Area,18386,9226.0,0.502,02050,18040.0,119.0,Bethel Census Area,Alaska,0.152778
4,02060,AK,Bristol Bay Borough,836,712.0,0.852,02060,890.0,4.0,Bristol Bay Borough,Alaska,0.127430


In [8]:
#look at NAs after join
df_rank.isna().sum()

fips                                   0
state                                  0
county                                 0
population                             0
actuals.vaccinationsCompleted         13
metrics.vaccinationsCompletedRatio    13
geo_id                                 2
total_pop                              2
black_pop                              2
county_name                           83
state_name                            83
vax_hes                               83
dtype: int64

In [9]:
#create variables used for scoring
df_rank['percent_black'] = df_rank['black_pop']/df_rank['total_pop']
df_rank['not_vax'] = 1 - df_rank['metrics.vaccinationsCompletedRatio']
df_rank['pop_scaled'] = (df_rank['population']-df_rank['population'].min())/(df_rank['population'].max()-df_rank['population'].min())

In [10]:
#read in and clean hbcu data
hbcu = pd.read_csv('hbcus.csv')
hbcu['state'] = hbcu['state_name'].map(us_state_abbrev)
hbcu = hbcu.groupby(['county_name','state'])['Name'].apply(list).reset_index(name = 'HBCUs')
#join hbcu data
df_rank = pd.merge(df_rank, hbcu, how='left', left_on=['county','state'], right_on=['county_name','state'])

In [11]:
#create a score
df_rank['score'] = df_rank['pop_scaled']*(df_rank[['percent_black', 'vax_hes', 'not_vax']].sum(1))/3

In [12]:
#filter out 0 score cause by NA and counties without a black pop
df_rank = df_rank[df_rank['score'] != 0]
df_rank = df_rank[df_rank['black_pop'] > 0]
#create ranking based on scores
df_rank['rank'] = df_rank['score'].rank(ascending = False)

In [13]:
#df by rankings
df_rank.sort_values('rank')

,fips,state,county,population,actuals.vaccinationsCompleted,metrics.vaccinationsCompletedRatio,geo_id,total_pop,black_pop,county_name_x,state_name,vax_hes,percent_black,not_vax,pop_scaled,county_name_y,HBCUs,score,rank
204,06037,CA,Los Angeles County,10039107,5267403.0,0.525,06037,10098052.0,795505.0,Los Angeles County,California,0.073247,0.078778,0.475,1.000000,NaN,NaN,0.209008,1.0
2703,48201,TX,Harris County,4713325,2048031.0,0.435,48201,4602523.0,855008.0,Harris County,Texas,0.135241,0.185769,0.565,0.469492,Harris County,[Texas Southern University],0.138658,2.0
709,17031,IL,Cook County,5150233,2692665.0,0.523,17031,5223719.0,1213706.0,Cook County,Illinois,0.092141,0.232345,0.477,0.513013,NaN,NaN,0.137058,3.0
178,04013,AZ,Maricopa County,4485414,1865852.0,0.416,04013,4253913.0,220519.0,Maricopa County,Arizona,0.155809,0.051839,0.584,0.446789,NaN,NaN,0.117900,4.0
2659,48113,TX,Dallas County,2635516,1105703.0,0.420,48113,2586552.0,572790.0,Dallas County,Texas,0.127341,0.221449,0.580,0.262519,Dallas County,[Paul Quinn College],0.081275,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,06003,CA,Alpine County,1129,688.0,0.609,06003,1146.0,9.0,Alpine County,California,0.082025,0.007853,0.391,0.000104,NaN,NaN,0.000017,3082.0
2824,48443,TX,Terrell County,776,313.0,0.403,48443,862.0,23.0,Terrell County,Texas,0.082975,0.026682,0.597,0.000069,NaN,NaN,0.000016,3083.0
284,08079,CO,Mineral County,769,308.0,0.401,08079,823.0,16.0,Mineral County,Colorado,0.086967,0.019441,0.599,0.000068,NaN,NaN,0.000016,3084.0
27,02282,AK,Yakutat City and Borough,579,371.0,0.641,02282,689.0,2.0,Yakutat City and Borough,Alaska,0.076858,0.002903,0.359,0.000049,NaN,NaN,0.000007,3085.0


In [18]:
#ranked counties with hbcus
hbcu_rank = df_rank[df_rank['HBCUs'].notna()].sort_values('rank')
hbcu_rank.head()

,fips,state,county,population,actuals.vaccinationsCompleted,metrics.vaccinationsCompletedRatio,total_pop,black_pop,vax_hes,percent_black,not_vax,pop_scaled,HBCUs,score,rank
2703,48201,TX,Harris County,4713325,2048031.0,0.435,4602523.0,855008.0,0.135241,0.185769,0.565,0.469492,[Texas Southern University],0.138658,2.0
2659,48113,TX,Dallas County,2635516,1105703.0,0.420,2586552.0,572790.0,0.127341,0.221449,0.580,0.262519,[Paul Quinn College],0.081275,5.0
362,12086,FL,Miami-Dade County,2716940,1582091.0,0.582,2715516.0,436864.0,0.119340,0.160877,0.418,0.270629,[Florida Memorial University],0.062986,12.0
2617,48029,TX,Bexar County,2003554,913067.0,0.456,1925865.0,136411.0,0.132460,0.070831,0.544,0.199568,[St. Philip's College],0.049712,17.0
2586,47157,TN,Shelby County,937166,339954.0,0.351,937005.0,499489.0,0.122912,0.533070,0.649,0.093344,[Le Moyne-Owen College],0.040604,22.0


In [15]:
#county containing case study university
df_rank[df_rank['county'] == 'Waller County']

,fips,state,county,population,actuals.vaccinationsCompleted,metrics.vaccinationsCompletedRatio,geo_id,total_pop,black_pop,county_name_x,state_name,vax_hes,percent_black,not_vax,pop_scaled,county_name_y,HBCUs,score,rank
2839,48473,TX,Waller County,55246,16072.0,0.291,48473,49987.0,12222.0,Waller County,Texas,0.172249,0.244504,0.709,0.005495,Waller County,[Prairie View A & M University],0.002062,755.0


In [21]:
df_rank = df_rank.drop(columns = ['geo_id', 'county_name_x', 'state_name', 'county_name_y'])

with pd.ExcelWriter('target_areas.xlsx') as writer:
    df_rank.sort_values('rank').to_excel(writer, sheet_name = 'All Counties', index = False)
    hbcu_rank.sort_values('rank').to_excel(writer, sheet_name = 'HBCU Counties', index = False)